# Step 1: Import necessary libraries

In [1]:
#import SQL library
import pyodbc
# Using implicit package from Cython 
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import implicit

In [2]:
#establishes an open connection to SQL
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-RQT0ED4\SQLEXPRESS;'
                      'Database=AdventureWorksDW2017;'
                      'Trusted_Connection=yes;')

# Step 2: Import data and clean data

In [3]:
#pull sales data from SQL including region and product for assosiation recommendations

pythonSQLobject=pd.read_sql_query("""
SELECT * FROM [dbo].[DATA_ALL5]""", conn)

#conn.close() # please close it after !

In [4]:
df = pythonSQLobject
df.head()

,CustomerKey,ProductKey,GeographyKey,SalesTerritoryKey,OrderDate,Year,Quarter,Month,Week,Day,...,Gender,YearlyIncome,TotalChildren,EnglishEducation,EnglishOccupation,HouseOwnerFlag,NumberCarsOwned,CommuteDistance,DateFirstPurchase,Age
0,11000,344,26,9,2011-01-19,2011,1,1,4,19,...,M,90000.0,2,Bachelors,Professional,1,0,1-2 Miles,2011-01-19,40
1,11000,353,26,9,2013-01-18,2013,1,1,3,18,...,M,90000.0,2,Bachelors,Professional,1,0,1-2 Miles,2011-01-19,40
2,11000,485,26,9,2013-01-18,2013,1,1,3,18,...,M,90000.0,2,Bachelors,Professional,1,0,1-2 Miles,2011-01-19,40
3,11000,573,26,9,2013-05-03,2013,2,5,18,3,...,M,90000.0,2,Bachelors,Professional,1,0,1-2 Miles,2011-01-19,40
4,11000,541,26,9,2013-05-03,2013,2,5,18,3,...,M,90000.0,2,Bachelors,Professional,1,0,1-2 Miles,2011-01-19,40


In [5]:
# Load the data in simialr fashion and keep the required columns, like User , MCAT and PURCHASE count
#raw_data = pd.read_csv('cat_13.csv')
#raw_data = df.loc[:,['CustomerKey','ProductKey','Product_Name','Sales_Volume']]
#raw_data = raw_data.drop(raw_data.columns[1], axis=1)
#raw_data.columns = ['user', 'item','Product', 'purchase_cnt']

In [6]:
#raw_data.head()

In [7]:
cleaned_retail = df.loc[pd.isnull(df.CustomerKey) == False]

In [8]:
cleaned_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60398 entries, 0 to 60397
Data columns (total 34 columns):
CustomerKey              60398 non-null int64
ProductKey               60398 non-null int64
GeographyKey             60398 non-null int64
SalesTerritoryKey        60398 non-null int64
OrderDate                60398 non-null datetime64[ns]
Year                     60398 non-null int64
Quarter                  60398 non-null int64
Month                    60398 non-null int64
Week                     60398 non-null int64
Day                      60398 non-null int64
Sales_Volume             60398 non-null int64
Transaction_Volume       60398 non-null object
Revenue                  60398 non-null float64
Product_Cost             60398 non-null float64
Profit                   60398 non-null float64
ProductCat               60398 non-null object
ProdctSubCat             60398 non-null object
Product_Name             60398 non-null object
SalesTerritoryGroup      60398 non-null obje

Much better. Now we have no missing values and all of the purchases can be matched to a specific customer.

Before we make any sort of ratings matrix, it would be nice to have a lookup table that keeps track of each item ID along with a description of that item. Let's make that now.

In [9]:
item_lookup = cleaned_retail[['ProductKey', 'Product_Name']].drop_duplicates() # Only get unique item/description pairs
#item_lookup['ProductKey'] = item_lookup.ProductKey.astype(str) # Encode as strings for future lookup ease
item_lookup['ProductKey'] = item_lookup.ProductKey.astype(int)

In [10]:
item_lookup.head()

,ProductKey,Product_Name
0,344,"Mountain-100 Silver, 38"
1,353,"Mountain-200 Silver, 38"
2,485,Fender Set - Mountain
3,573,"Touring-1000 Blue, 46"
4,541,Touring Tire


In [11]:
item_lookup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 20385
Data columns (total 2 columns):
ProductKey      158 non-null int32
Product_Name    158 non-null object
dtypes: int32(1), object(1)
memory usage: 3.1+ KB


This can tell us what each item is, such as that ProductKey 541 is a Touring Tire. Now that this has been created, we need to:

Group purchase quantities together by ProductKey and item ID
Change any sums that equal zero to one (this can happen if items were returned, but we want to indicate that the user actually purchased the item instead of assuming no interaction between the user and the item ever took place)
Only include customers with a positive purchase total to eliminate possible errors
Set up our sparse ratings matrix
This last step is especially important if you don't want to have unnecessary memory issues! If you think about it, our matrix is going to contain thousands of items and thousands of users with a user/item value required for every possible combination. That is a LARGE matrix, so we can save a lot of memory by keeping the matrix sparse and only saving the locations and values of items that are not zero.

The code below will finish the preprocessing steps necessary for our final ratings sparse matrix:

In [12]:
cleaned_retail['CustomerKey'] = cleaned_retail.CustomerKey.astype(int) # Convert to int for customerKey
cleaned_retail = cleaned_retail[['ProductKey', 'Sales_Volume', 'CustomerKey']] # Get rid of unnecessary info
grouped_cleaned = cleaned_retail.groupby(['CustomerKey', 'ProductKey']).sum().reset_index() # Group together
grouped_cleaned.Sales_Volume.loc[grouped_cleaned.Sales_Volume == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_purchased = grouped_cleaned.query('Sales_Volume > 0') # Only get customers where purchase totals were positive

If we look at our final resulting matrix of grouped purchases, we see the following:

In [13]:
grouped_purchased.head()

,CustomerKey,ProductKey,Sales_Volume
0,11000,214,1
1,11000,344,1
2,11000,353,1
3,11000,485,1
4,11000,488,1


In [14]:
grouped_purchased.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59051 entries, 0 to 59050
Data columns (total 3 columns):
CustomerKey     59051 non-null int64
ProductKey      59051 non-null int64
Sales_Volume    59051 non-null int64
dtypes: int64(3)
memory usage: 1.8 MB


Instead of representing an explicit rating, the purchase quantity can represent a "confidence" in terms of how strong the interaction was. Items with a larger number of purchases by a customer can carry more weight in our ratings matrix of purchases.

Our last step is to create the sparse ratings matrix of users and items utilizing the code below:

In [15]:
customers = list(np.sort(grouped_purchased.CustomerKey.unique())) # Get our unique customers
products = list(grouped_purchased.ProductKey.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Sales_Volume) # All of our purchases

rows = grouped_purchased.CustomerKey.astype("category", categorie=customers).cat.codes 
# Get the associated row indices
cols = grouped_purchased.ProductKey.astype("category", categorie=products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

Let's check our final matrix object:

In [16]:
purchases_sparse

<18484x158 sparse matrix of type '<class 'numpy.int32'>'
	with 59051 stored elements in Compressed Sparse Row format>

##### We have 18484 customers with 158 items. For these user/item interactions, 59051 of these items had a purchase.

In [17]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

97.97803231806365

97.97% of the interaction matrix is sparse. For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. We are well below this, so we should be able to get decent results.

# Part 3: Creating a Training and Validation Set

Typically in Machine Learning applications, we need to test whether the model we just trained is any good on new data it hasn't yet seen before from the training phase. We do this by creating a test set completely separate from the training set. Usually this is fairly simple: just take a random sample of the training example rows in our feature matrix and separate it away from the training set. That normally looks like this:

With collaborative filtering, that's not going to work because you need all of the user/item interactions to find the proper matrix factorization. A better method is to hide a certain percentage of the user/item interactions from the model during the training phase chosen at random. Then, check during the test phase how many of the items that were recommended the user actually ended up purchasing in the end. Ideally, you would ultimately test your recommendations with some kind of A/B test or utilizing data from a time series where all data prior to a certain point in time is used for training while data after a certain period of time is used for testing.

For this example, because the time period is only 8 months and because of the purchasing type (products), it is most likely products won't be purchased again in a short time period anyway. This will be a better test. You can see an example here:

Our test set is an exact copy of our original data. The training set, however, will mask a random percentage of user/item interactions and act as if the user never purchased the item (making it a sparse entry with a zero). We then check in the test set which items were recommended to the user that they ended up actually purchasing. If the users frequently ended up purchasing the items most recommended to them by the system, we can conclude the system seems to be working.

As an additional check, we can compare our system to simply recommending the most popular items to every user (beating popularity is a bit difficult). This will be our baseline.

This method of testing isn't necessarily the "correct" answer, because it depends on how you want to use the recommender system. However, it is a practical way of testing performance I will use for this example.

Now that we have a plan on how to separate our training and testing sets, let's create a function that can do this for us. We will also import the random library and set a seed so that you will see the same results as I did.

In [18]:
import random

In [19]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered 

This will return our training set, a test set that has been binarized to 0/1 for purchased/not purchased, and a list of which users had at least one item masked. We will test the performance of the recommender system on these users only. I am masking 20% of the user/item interactions for this example.

In [20]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

Now that we have our train/test split, it is time to implement the alternating least squares algorithm from the Hu, Koren, and Volinsky paper.

# Part 4: Implementing ALS for Implicit Feedback

In [21]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of ratings with shape m x n, where m is the number of users and n is the number of items.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various ratings.
    
    iterations - The number of times to alternate between both user feature vector and item feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the user/item feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for users and items. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, which is just CuI since we never added 1
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, which is just CiI since we never added 1
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

Hopefully the comments are enough to see how the code was structured. You want to keep the matrices sparse where possible to avoid memory issues! Let's try just a single iteration of the code to see how it works (it's pretty slow right now!) I will choose 20 latent factors as my rank matrix size along with an alpha of 15 and regularization of 0.1 (which I found in testing does the best). 

In [22]:
user_vecs, item_vecs = implicit_weighted_ALS(product_train, lambda_val = 0.1, alpha = 15, iterations = 10,rank_size = 20)

We can investigate ratings for a particular user by taking the dot product between the user and item vectors (U and V). Let's look at our first user.

In [23]:
user_vecs[0,:].dot(item_vecs).toarray()[0,:5]

array([ 0.86173063,  0.01845375, -0.06330339, -0.46644997, -0.01004098])

This is a sample of the first five items out of the 3664 in our stock. The first user in our matrix has the fifth item with the greatest recommendation out of the first five items. However, notice we only did one iteration because our algorithm was so slow! You should iterate at least ten times according to the authors so that U and V converge. We could wait 15 minutes to let this run, or . . . use someone else's code that is much faster!

## Speeding Up ALS
This code in its raw form is just too slow. We have to do a lot of looping, and we haven't taken advantage of the fact that our algorithm is embarrasingly parallel, since we could do each iteration of the item and user vectors independently. Fortunately, as I was still finishing this up, Ben Frederickson at Flipboard had perfect timing and came out with a version of ALS for Python utilizing Cython and parallelizing the code among threads. You can read his blog post about using it for finding similar music artists using matrix factorization here and his implicit library here. He claims it is even faster than Quora's C++ QMF, but I haven't tried theirs. All I can tell you is that it is over 1000 times faster than this bare bones pure Python version when I tested it. Install this library before you continue and follow the instructions. If you have conda installed, just do pip install implicit and you should be good to go.

First, import his library so we can utilize it for our matrix factorization

In [24]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead


### Evaluating the Recommender System

Remember that our training set had 20% of the purchases masked? This will allow us to evaluate the performance of our recommender system. Essentially, we need to see if the order of recommendations given for each user matches the items they ended up purchasing. A commonly used metric for this kind of problem is the area under the Receiver Operating Characteristic (or ROC) curve. A greater area under the curve means we are recommending items that end up being purchased near the top of the list of recommended items. Usually this metric is used in more typical binary classification problems to identify how well a model can predict a positive example vs. a negative one. It will also work well for our purposes of ranking recommendations.

In order to do that, we need to write a function that can calculate a mean area under the curve (AUC) for any user that had at least one masked item. As a benchmark, we will also calculate what the mean AUC would have been if we had simply recommended the most popular items. Popularity tends to be hard to beat in most recommender system problems, so it makes a good comparison.

First, let's make a simple function that can calculate our AUC. Scikit-learn has one we can alter a bit.

In [25]:
from sklearn import metrics

In [26]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

Now, utilize this helper function inside of a second function that will calculate the AUC for each user in our training set that had at least one item masked. It should also calculate AUC for the most popular items for our users to compare.

In [27]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

We can now use this function to see how our recommender system is doing. To use this function, we will need to transform our output from the ALS function to csr_matrix format and transpose the item vectors. The original pure Python version output the user and item vectors into the correct format already.

In [28]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.772, 0.836)

We can see that our recommender system could not beat popularity. Our system had a mean AUC of 0.77, while the popular item benchmark had a higher AUC of 0.836. You can go back and tune the hyperparameters if you wish to see if you can get a higher AUC score. Ideally, you would have separate train, cross-validation, and test sets so that you aren't overfitting while tuning the hyperparameters, but this setup is adequate to demonstrate how to check that the system is working.

# A Recommendation Example
We now have our recommender system trained and have proven it beats the benchmark of popularity. An AUC of 0.87 means the system is recommending items the user in fact had purchased in the test set far more frequently than items the user never ended up purchasing. To see an example of how it works, let's examine the recommendations given to a particular user and decide subjectively if they make any sense.

First, however, we need to find a way of retrieving the items already purchased by a user in the training set. Initially, we will create an array of our customers and items we made earlier.

In [29]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

Now, we can create a function that will return a list of the item descriptions from our earlier created item lookup table.

In [30]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup.ProductKey.isin(prod_codes)]

We need to look these up by a customer's ID. Looking at the list of customers:

In [31]:
customers_arr[:5]

array([11000, 11001, 11002, 11003, 11004], dtype=int64)

In [32]:
products_arr[:5]

array([214, 344, 353, 485, 488], dtype=int64)

we can see that the first customer listed has an ID of 11000. Let's examine their purchases from the training set.

In [33]:
get_items_purchased(11000, product_train, customers_arr, products_arr, item_lookup)

,ProductKey,Product_Name
6,214,"Sport-100 Helmet, Red"
129,535,LL Mountain Tire
2719,362,"Mountain-200 Black, 46"
3138,581,"Road-350-W Yellow, 42"
7090,372,"Road-250 Red, 58"
13993,386,"Road-550-W Yellow, 42"


We can see that the customer purchased Bike and bike related item. What items does the recommender system say this customer should purchase? We need to create another function that does this. Let's also import the MinMaxScaler from scikit-learn to help with this.

In [34]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 3):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.Product_Name.loc[item_lookup.ProductKey == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'ProductKey': codes, "Product_Name" : descriptions}) # Create a dataframe 
    return final_frame[['ProductKey', 'Product_Name']] # Switch order of columns around

Essentially, this will retrieve the N highest ranking dot products between our user and item vectors for a particular user. Items already purchased are not recommended to the user. For now, let's use a default of 10 items and see what the recommender system decides to pick for our customer.

In [35]:
rec_items(11000, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,num_items = 3)

,ProductKey,Product_Name
0,571,"Touring-3000 Yellow, 58"
1,578,"Touring-2000 Blue, 50"
2,472,"Classic Vest, M"


These recommendations seem quite good! Remember that the recommendation system has no real understanding of what a Bike product is. All it knows is the purchase history. It identified that people purchasing Bike product may also want to purchase bike of a differing use. The recommender system also suggests Hydration Pack, which is bike rider need. I personally was blown away by how well the system seems to pick up on these sorts of shopping patterns. Let's try another user that hasn't made a large number of purchases.

In [36]:
get_items_purchased(29480, product_train, customers_arr, products_arr, item_lookup)

,ProductKey,Product_Name
2,485,Fender Set - Mountain
1773,592,"Mountain-500 Silver, 42"
1820,374,"Road-250 Black, 44"
7847,388,"Road-550-W Yellow, 44"


In [37]:
rec_items(29480, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)

,ProductKey,Product_Name
0,382,"Road-550-W Yellow, 38"
1,571,"Touring-3000 Yellow, 58"
2,314,"Road-150 Red, 56"


It certainly picked up on the bike theme along with accessories items. Again, these recommendations seem very impressive given the system doesn't understand the content behind the recommendations. Let's try one more.

In [38]:
get_items_purchased(12000, product_train, customers_arr, products_arr, item_lookup)

,ProductKey,Product_Name
127,228,"Long-Sleeve Logo Jersey, S"
1773,592,"Mountain-500 Silver, 42"
1815,378,"Road-250 Black, 52"
2861,354,"Mountain-200 Silver, 42"
3138,581,"Road-350-W Yellow, 42"
7090,372,"Road-250 Red, 58"
7847,388,"Road-550-W Yellow, 44"


This customer seems like they are buying products suitable for bike and accessories. What other items does the recommender system think they might like?

In [39]:
rec_items(12000, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 3)

,ProductKey,Product_Name
0,356,"Mountain-200 Silver, 46"
1,382,"Road-550-W Yellow, 38"
2,374,"Road-250 Black, 44"


Once again, the recommender system comes through! Definitely a lot of Component and accessories related items in this recommendation list.

In [46]:
# Lets find out average Sales Amount for customers
df.SalesAmount.sum()

AttributeError: 'DataFrame' object has no attribute 'SalesAmount'

In [42]:
# Lets find out average Sales Amount for customers
df.Sales_Volume.mean()

1.0

In [43]:
df.groupby(['CustomerKey']).agg({'Sales_Volume':'sum'})

,Sales_Volume
CustomerKey,
11000,8
11001,11
11002,4
11003,9
11004,6
...,...
29479,1
29480,5
29481,1


In [45]:
df.Sales_Volume.sum()

60398

In [44]:
df.Transaction_Volume.count() 

60398